## libraries

In [23]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torchviz import make_dot
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

- Make the model determinastic (sets the seed for PyTorch's random number generator to 0, ensuring reproducibility in random processes.)

In [2]:
_ = torch.manual_seed(0)

#### we will be training a network to classify MNIST digits and fine-tune the network on a particular digit on which model is not performed well

transformation includes <br>
- converting the image into a tensor
- Normalizing the tensor with a mean of 0.1307 and standard deviation 0.3081
- These specific values are choosen based on the statistics of the MNIST dataset (explained later)
- We can use mean and standard deviation of the standard dataset
- The purpose of normalization is to scale the pixel values so that they have a similar scale

In [ ]:
#creates a sequence of image transformations, normalizes the image to have a mean of 0.1307 and a standard deviation of 0.3081
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])


In [13]:
# Load MNIST dataset
mnist_trainset = datasets.MNIST(root = './data', train=True, download=True, transform=transforms)

# create a loader
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=64, shuffle=True)

In [12]:
# Calculate the mean and standard deviation for the MNIST dataset
mnist_mean = mnist_trainset.data.float().mean() / 255
mnist_std = mnist_trainset.data.float().std() / 255

print("Mean:", mnist_mean)
print("Standard Deviation:", mnist_std)


Mean: tensor(0.1307)
Standard Deviation: tensor(0.3081)


In [14]:
# Load the mnist test dataset
mnist_testset = datasets.MNIST(root = './data', train=False, download=True, transform=transforms)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=64, shuffle=True)

In [16]:
# define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

#### Create an overly expensive neural network to classify MNIST digits

In [18]:
class ExpensiveNet(nn.Module):
    def __init__(self, hidden_size_1 = 1000, hidden_size_2 = 2000):
        super(ExpensiveNet, self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x
net = ExpensiveNet().to(device)

In [24]:
# Dummy input for visualization
dummy_input = torch.randn(1, 1, 28, 28).to(device)

# Visualize the network
make_dot(net(dummy_input), params=dict(net.named_parameters()))

# Create a graph of the neural network
graph = make_dot(net(dummy_input), params=dict(net.named_parameters()))

# Save the graph as an image (optional)
graph.render(filename='expensive_net', format='png', cleanup=True)

# Display the graph (optional)
graph.view()

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH